In [1]:
import json
import requests
import datetime
import time
import pytz
import pandas as pd
import pprint as pp
import matplotlib.pyplot as plt

In [14]:
# This cell is tagged `parameters`

config_file = '/home/emily/Desktop/projects/test/badass-data-science/badassdatascience/forex/data/DEVELOPMENT.json'
count = 5000 
granularity = 'D'
now = True

#instruments = 'EUR_USD,USD_CAD,USD_JPY,USD_CHF,AUD_USD,GBP_USD,NZD_USD'
instrument = 'EUR_USD'

price_types = 'BAM'
error_retry_interval = 5

In [4]:
print(count)
print(granularity)

5000
D


In [17]:
def get_oanda_headers(config):
    headers = {
        'Content-Type' : 'application/json',
        'Authorization' : 'Bearer ' + config['token'],
        'Accept-Datetime-Format' : config['oanda_date_time_format'],
    }
    return headers

In [10]:
if True:
    def deal_with_candlestick_format_and_time(candle, timezone):
        candle['time'] = int(float(candle['time']))
        time_dt = datetime.datetime.fromtimestamp(candle['time'], tz = timezone)
        candle['time_iso'] = time_dt.isoformat()
        candle['weekday'] = time_dt.weekday()
        candle['hour'] = time_dt.hour

        #
        # deal with prices that are currently string values but need to be float
        #
        # and reorganize them
        #
        for price_type in ['bid', 'mid', 'ask']:  # what about BAM ?
            for candlestick_component in candle[price_type].keys():
                candle[price_type + '_' + candlestick_component] = float(candle[price_type][candlestick_component])
            candle[price_type + '_return'] = candle[price_type + '_c'] - candle[price_type + '_o']
            candle[price_type + '_volatility'] = candle[price_type + '_h'] - candle[price_type + '_l']
            
        for price_type in ['bid', 'mid', 'ask']:
            del(candle[price_type])
            
        #return None

In [43]:
# no previous

class test():
    def __init__(
        self,
        config_file,
        count,
        granularity,
        now,
        instrument,
        price_types,
        error_retry_interval = 5,
        keep_complete_only = True
    ):

        # command line arguments
        self.config_file = config_file
        self.count = count
        self.granularity = granularity
        self.now = now
        self.instrument = instrument
        self.price_types = price_types
        self.error_retry_interval = error_retry_interval
        self.keep_complete_only = keep_complete_only

        # initialize
        self.timezone_to_use = 'America/Toronto'   # Don't change this!    # not sure we are using this... check later
        self.end_date = None
        self.output_file_sans_extension = 'output/' + granularity
        self.start_time = datetime.datetime(2010, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc).timestamp()
        self.timezone = pytz.timezone(self.timezone_to_use)  # not sure we are using this... check later


        self.end_date_original = int(time.mktime(datetime.datetime.now().timetuple()))
        

    def load_config(self):
        with open(self.config_file) as f:
            self.config = json.load(f)

    def get_headers(self):
        self.headers = get_oanda_headers(self.config)


    def get_instrument_candlesticks(self, end_date):
        url = (
            self.config['server']
            + '/v3/instruments/' + self.instrument
            + '/candles?count=' + str(self.count)
            + '&price=' + self.price_types
            + '&granularity=' + self.granularity
            + '&to=' + str(end_date)
        )
        
        worked = False
        while not worked:
            try:
                r = requests.get(url, headers = self.headers)
                worked = True
            except:
                time.sleep(error_retry_interval)
        
        rj = r.json()
        return rj





    def booger(self):
        
        finished = False
        end_date = self.end_date_original

        self.insert_many_list = []

        # loop through the timestamp ranges for each set of n=count values
        while not finished:

            # retrieve the instrument candlesticks from the Oanda server
            rj = self.get_instrument_candlesticks(end_date) # instrument, count, price_types, granularity, end_date)        
            candlesticks = rj['candles']
            
            # deal with timestamps and time-related content
            date_list = []
            for candle in candlesticks:
                deal_with_candlestick_format_and_time(candle, self.timezone)
                date_list.append(candle['time'])
                
            rj['timestamp_int_min'] = min(date_list)
            rj['timestamp_int_max'] = max(date_list)


            for candle in rj['candles']:
                candle['instrument'] = self.instrument
                candle['granularity'] = self.granularity

                if self.keep_complete_only:
                    if candle['complete']:    
                        self.insert_many_list.append(candle)
                else:
                    self.insert_many_list.append(candle)
            

            # Are we done?
            if len(date_list) < count:
                finished = True

            if min(date_list) < self.start_time:
                finished = True
            
            # prepare for the next iteration
            end_date = rj['timestamp_int_min'] - 0.1


    def fit(self):
        self.load_config()
        self.get_headers()
        self.booger()
        

In [44]:
t = test(config_file, count, granularity, now, instrument, price_types, error_retry_interval = 5)
t.fit()

In [45]:
pp.pprint(t.insert_many_list[0:3])

[{'ask_c': 1.41837,
  'ask_h': 1.41934,
  'ask_l': 1.41837,
  'ask_o': 1.41934,
  'ask_return': -0.0009700000000001374,
  'ask_volatility': 0.0009700000000001374,
  'bid_c': 1.41737,
  'bid_h': 1.41834,
  'bid_l': 1.41737,
  'bid_o': 1.41834,
  'bid_return': -0.0009699999999999154,
  'bid_volatility': 0.0009699999999999154,
  'complete': True,
  'granularity': 'D',
  'hour': 17,
  'instrument': 'EUR_USD',
  'mid_c': 1.41787,
  'mid_h': 1.41884,
  'mid_l': 1.41787,
  'mid_o': 1.41884,
  'mid_return': -0.0009700000000001374,
  'mid_volatility': 0.0009700000000001374,
  'time': 1192222800,
  'time_iso': '2007-10-12T17:00:00-04:00',
  'volume': 3,
  'weekday': 4},
 {'ask_c': 1.4181,
  'ask_h': 1.41862,
  'ask_l': 1.4171,
  'ask_o': 1.41862,
  'ask_return': -0.000520000000000076,
  'ask_volatility': 0.0015199999999999658,
  'bid_c': 1.4171,
  'bid_h': 1.41725,
  'bid_l': 1.4161,
  'bid_o': 1.41712,
  'bid_return': -1.999999999990898e-05,
  'bid_volatility': 0.0011499999999999844,
  'complet